In [23]:
import os, sys 
import pandas as pd
import numpy as np
os.chdir('../src')
import config
import glob
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
from sklearn.metrics import accuracy_score
import joblib
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
import IPython.display as ipd
from utils import utils
from sklearn import svm
from tqdm import tqdm

In [58]:
nn_folder = 'nn_11'

le = joblib.load(config.OUTPUT_FOLDER + '%s/label_encoder.dump'%nn_folder)

gt = {}
sub = {}
for flag  in  ['train', 'test']:
    gt[flag] = pd.read_csv(config.DATA_FOLDER + 'gt_%s.csv'%flag)

    sub[flag] = pd.read_csv(config.OUTPUT_FOLDER + '%s/pred_%s.csv'%(nn_folder, flag))
    sub[flag].rename(columns = {str(i) : le.inverse_transform(i) for i in np.arange(31)}, inplace = True)
#     sub[flag]['pred_target'] = sub_train.drop('id', 1).idxmax(axis =1)
    
    sub[flag] = gt[flag].merge(sub[flag], on = 'id')
    assert len(sub[flag]) == len(gt[flag])


In [59]:
sub['train'].head()

,id,path,person_id,target,bed,bird,cat,dog,down,eight,...,silence,six,stop,three,tree,two,up,wow,yes,zero
0,2fa39636_nohash_2_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,2fa39636,wow,5.352477e-16,3.461144e-15,2.294164e-19,1.176089e-15,2.249162e-11,2.872871e-16,...,1.054677e-21,1.477462e-21,1.292939e-17,2.827425e-19,7.110773e-20,2.242060e-15,2.292398e-18,1.000000,1.530969e-16,1.339706e-16
1,fb7c9b3b_nohash_1_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,fb7c9b3b,wow,1.283282e-08,2.561713e-10,2.526582e-12,8.243578e-11,8.160117e-08,3.817316e-13,...,4.434752e-14,1.097643e-14,1.217417e-10,2.662846e-12,7.933809e-13,1.938277e-11,3.875588e-11,0.999964,1.669085e-10,7.865300e-09
2,73af0c50_nohash_1_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,73af0c50,wow,5.151094e-05,1.168977e-05,5.400969e-06,8.056250e-04,3.891960e-03,6.743834e-06,...,2.379873e-06,1.300277e-06,3.737993e-04,1.091063e-06,7.226755e-07,7.282402e-05,3.943627e-04,0.980321,1.176353e-06,4.369192e-04
3,9e6bb505_nohash_1_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,9e6bb505,wow,3.106127e-13,7.612288e-14,5.053974e-15,7.275800e-12,4.564326e-09,1.196906e-12,...,2.226677e-19,3.000489e-16,8.490479e-13,1.220396e-15,1.346315e-16,2.988240e-12,2.283704e-14,0.999997,2.004664e-10,6.183005e-13
4,6ac35824_nohash_0_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,6ac35824,wow,1.663460e-05,1.911783e-06,3.147234e-07,5.439843e-09,3.245779e-05,1.846840e-08,...,1.703160e-08,9.379524e-11,3.245449e-07,1.995240e-08,4.448446e-11,4.757614e-10,4.949696e-07,0.999897,3.024122e-10,6.966250e-08


In [5]:
def get_splits(folder):
    splits = {}
    for i in np.arange(5):
        splits[i] = {}
        for c in ['train', 'val']:
            splits[i][c] = utils.read_list_from_file(config.OUTPUT_FOLDER + folder + '/splits/%s_%d.txt'%(c, i))
    return splits
splits_dict = get_splits(nn_folder)

In [82]:
clfs = {}
for c in tqdm(config.mapping_dict.keys()):
    clfs[c] = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1, random_state = 2018, max_iter = -1)
    clfs[c].fit(sub['train'][sub['train']['target'] == c][config.mapping_dict.keys()])
clfs['all'] = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1, random_state = 2018, max_iter = -1)
clfs['all'].fit(sub['train'][config.mapping_dict.keys()])

100%|██████████| 31/31 [00:01<00:00, 16.12it/s]


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=2018, shrinking=True, tol=0.001,
      verbose=False)

In [83]:
svm_df = {}
for flag in ['train', 'test']:
    svm_df[flag] = sub[flag][['id', 'path']].copy()
    print('Predicting for %s'%flag)
    for c in tqdm(config.mapping_dict.keys() + ['all']):
        svm_df[flag][c] = clfs[c].predict(sub[flag][config.mapping_dict.keys()])
        
    svm_df[flag]['max_label'] = svm_df[flag][config.mapping_dict.keys() + ['all']].max(axis = 1)
    

  0%|          | 0/32 [00:00<?, ?it/s]

Predicting for train


  0%|          | 0/32 [00:00<?, ?it/s]

Predicting for test


100%|██████████| 32/32 [01:57<00:00, 18.89s/it]


In [86]:
svm_df['test']['max_label'].value_counts()

 1    158024
-1       514
Name: max_label, dtype: int64

In [71]:
svm_df['train'].head()

,id,path,sheila,seven,right,up,house,one,four,zero,...,off,tree,dog,bed,cat,nine,five,silence,left,max_label
0,2fa39636_nohash_2_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
1,fb7c9b3b_nohash_1_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
2,73af0c50_nohash_1_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,9e6bb505_nohash_1_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
4,6ac35824_nohash_0_wow,/home/tony/ssd_buffer/kaggle_speech/train/audi...,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [85]:
N = 1245
item = svm_df['train'][svm_df['train']['max_label'] == -1]
path = item['path'].values[N] 
target = item['id'].values[N].split('_')[-1]
print(target)
ipd.Audio(path)

on
